In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
import joblib
import re
import os

# 1.Data Preprocessing

In [2]:
# 1. Load dataset
df = pd.read_csv('data/processed_data.csv')


# 2. Combine subject and message, then clean text data
def clean_text(text):
    text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [3]:
# Combine subject and message columns
df['combined_text'] = (df['subject'].fillna('') + ' ' + df['message'].fillna('')).apply(clean_text)


# 3. Feature extraction (TF-IDF)
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X = vectorizer.fit_transform(df['combined_text'])

# 4. Target variable
y = df['label']


In [5]:
# 4. Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# 2. Model Training

In [6]:
# 5. Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


# 6. Evaluate model
y_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(y_test, y_pred))
print('Precision:', precision_score(y_test, y_pred))
print('Recall:', recall_score(y_test, y_pred))
print('F1 Score:', f1_score(y_test, y_pred))
print('\nClassification Report:\n', classification_report(y_test, y_pred))

Accuracy: 0.9917130734553169
Precision: 0.9911565977742448
Recall: 0.9964039556487864
F1 Score: 0.9937733499377335

Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.98      0.99      5073
           1       0.99      1.00      0.99     10011

    accuracy                           0.99     15084
   macro avg       0.99      0.99      0.99     15084
weighted avg       0.99      0.99      0.99     15084



In [7]:

# 7. Save model and vectorizer
os.makedirs('model', exist_ok=True)
joblib.dump(model, 'model/spam_classifier.joblib')
joblib.dump(vectorizer, 'model/tfidf_vectorizer.joblib')

print('Model and vectorizer saved in /model directory.')


Model and vectorizer saved in /model directory.
